In [1]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [2]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [3]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [2]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [3]:
def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = file.readlines()
    # Remove leading and trailing whitespaces, and filter out empty lines
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

**ENGLISH TO HINDI**

In [9]:
# Define the path to your input text file containing sentences
input_file_path = "/content/test.en_1000.txt"
# Define the path for the output file
output_file_path = "indicTRANS_eng_to_hin.txt"

# Read sentences from the input file
en_sents = read_sentences_from_file(input_file_path)

# Rest of your code remains unchanged from here
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

# Save translations to the output file
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for input_sentence, translation in zip(en_sents, hi_translations):
        output_file.write(f"{translation}\n")
    print(f"Translations saved to {output_file_path}")

# Flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

Translations saved to indicTRANS_eng_to_hin.txt


**HINDI TO ENGLISH**

In [7]:
# Define the path to your input text file containing sentences
input_file_path = "/content/test.hi_1000.txt"
# Define the path for the output file
output_file_path = "indicTRANS_hin_to_eng.txt"

# Read sentences from the input file
hi_sents = read_sentences_from_file(input_file_path)

# Rest of your code remains unchanged from here
hi_indic_ckpt_dir = "ai4bharat/indictrans2-indic-en-dist-200M"
hi_indic_tokenizer, hi_indic_model = initialize_model_and_tokenizer(hi_indic_ckpt_dir, "indic-en", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "hin_Deva","eng_Latn"
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, hi_indic_model, hi_indic_tokenizer, ip)

# Save translations to the output file
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for input_sentence, translation in zip(hi_sents, en_translations):
        output_file.write(f"{translation}\n")
    print(f"Translations saved to {output_file_path}")

# Flush the models to free the GPU memory
del hi_indic_tokenizer, hi_indic_model


Translations saved to indicTRANS_hin_to_eng.txt


**HINDI TO MARATHI**

In [11]:
# Define the path to your input text file containing sentences
input_file_path = "/content/test.hi_1000.txt"
# Define the path for the output file
output_file_path = "indicTRANS_hin_to_mar.txt"

# Read sentences from the input file
hi_sents = read_sentences_from_file(input_file_path)

# Rest of your code remains unchanged from here
hi_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"
hi_indic_tokenizer, hi_indic_model = initialize_model_and_tokenizer(hi_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "hin_Deva","mar_Deva"
mar_translations = batch_translate(hi_sents, src_lang, tgt_lang, hi_indic_model, hi_indic_tokenizer, ip)

# Save translations to the output file
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for input_sentence, translation in zip(hi_sents, mar_translations):
        output_file.write(f"{translation}\n")
    print(f"Translations saved to {output_file_path}")

# Flush the models to free the GPU memory
del hi_indic_tokenizer, hi_indic_model


Translations saved to indicTRANS_hin_to_mar.txt


**MARATHI TO HINDI**

In [8]:
# Define the path to your input text file containing sentences
input_file_path = "/content/test.mr_1000.txt"
# Define the path for the output file
output_file_path = "indicTRANS_mar_to_hin.txt"

# Read sentences from the input file
mr_sents = read_sentences_from_file(input_file_path)

# Rest of your code remains unchanged from here
mr_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"
mr_indic_tokenizer, mr_indic_model = initialize_model_and_tokenizer(mr_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "mar_Deva","hin_Deva"
hi_translations = batch_translate(mr_sents, src_lang, tgt_lang, mr_indic_model, mr_indic_tokenizer, ip)

# Save translations to the output file
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for input_sentence, translation in zip(mr_sents, hi_translations):
        output_file.write(f"{translation}\n")
    print(f"Translations saved to {output_file_path}")

# Flush the models to free the GPU memory
del mr_indic_tokenizer, mr_indic_model


config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Translations saved to indicTRANS_mar_to_hin.txt
